# 데이터 이해와 전처리

**이 노트북의 목표**: "내 직업이 AI에 얼마나 노출될까?"를 계산하기 위한 **데이터 준비하기**

## 필요한 데이터 2가지

1. 🤖 **MTurk AI 매핑 데이터** (mturk_mapping_matrix.csv)
   - **52개 능력**이 AI에 얼마나 영향받는지 (**Beta 값**)
   - 0에 가까우면: AI가 못함 (예: 물리적 힘)
   - 1에 가까우면: AI가 잘함 (예: 음성인식)

2. 📊 **O*NET 능력 데이터** (Abilities.xlsx)
   - **894개 직업**이 **52개 능력**을 얼마나 필요로 하는지
   - **Importance (중요도)**: 1-5점 척도
   - **Level (수준)**: 0-7점 척도
   - **Title**: 직업명 (직업 코드 변환용)


**이번 시간**: 2개 데이터를 하나로 병합하기!

---

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 데이터 경로 설정
RAW_DIR = '../datas/raw'
PROCESSED_DIR = '../datas/processed'

## 1. MTurk AI 매핑 데이터 이해하기

### MTurk 데이터란?
```
MTurk(Amazon Mechanical Turk)는 아마존이 운영하는 크라우드소싱 플랫폼입니다.
연구자들은 이 플랫폼을 통해 많은 사람들에게 설문조사를 의뢰했고,
"각 AI 응용 프로그램이 52개 능력을 얼마나 대체할 수 있는가?"를 평가받았습니다.
```

In [ ]:
# 실습: 데이터의 전체 구조를 확인하세요

In [ ]:
# 실습: 데이터의 마지막 5행을 확인하세요

In [ ]:
# 실습: 데이터의 기술 통계를 확인하세요

In [ ]:
# 실습: 'applications' 컬럼의 고유값 개수를 확인하세요

### MTurk 데이터 구조

**데이터 구성:**
- 10개 AI 응용 프로그램 × 52개 능력
- 각 셀: AI가 해당 능력을 얼마나 잘 수행하는지 (0~1 점수, Beta 값)
  - 0에 가까우면: AI가 못함 (예: 신체적 힘)
  - 1에 가까우면: AI가 잘함 (예: 텍스트 이해)

**우리가 사용할 데이터:**
- 우리는 이 중 **"Language Modeling(언어 모델)" 행만** 사용합니다
- 왜냐하면 ChatGPT, Gemini, Claude 등 최근 주목받는 AI가 바로 언어 모델이기 때문입니다

In [ ]:
# 실습: Language Modeling 응용 프로그램의 Beta 값 추출

In [ ]:
# 실습: Beta 값을 DataFrame으로 변환하여 정렬

In [ ]:
# 실습: AI가 가장 잘하는 능력 TOP 5를 확인하세요

In [ ]:
# 실습: AI가 가장 어려워하는 능력 TOP 5를 확인하세요

## 2. O*NET 2024 능력 데이터 이해하기

### O*NET이란?

**O*NET (Occupational Information Network)**
- 미국 노동부가 관리하는 **세계 최대 규모의 직업 정보 데이터베이스**
- 1,000개 가까운 직업에 대한 상세 정보 제공
- 정기적으로 업데이트 (우리는 2024년 7월 공개된 **버전 30.0** 사용)

**O*NET이 수집하는 정보:**
- 각 직업이 필요로 하는 능력(Abilities)
- 필요한 지식(Knowledge)
- 사용하는 기술(Skills)
- 업무 활동(Work Activities)
- 업무 환경(Work Context)
- ... 등 수십 가지 정보

**우리가 사용할 데이터: Abilities.xlsx**
- 각 직업이 **52개 능력**을 얼마나 필요로 하는지
- 두 가지 척도로 측정:
  - **Importance (중요도)**: 이 능력이 얼마나 중요한가?
  - **Level (수준)**: 이 능력을 얼마나 높은 수준으로 필요로 하는가?

먼저 데이터를 불러와서 전체 구조를 파악해보겠습니다.

In [ ]:
# O*NET 2024 능력 데이터 로드
onet_abil = pd.read_excel(f'{RAW_DIR}/Abilities.xlsx')

In [ ]:
# 실습: 데이터의 전체 구조를 확인하세요

In [ ]:
# 실습: 데이터의 처음 2행을 확인하세요

### O*NET Abilities 데이터 구조

**핵심 구조:**
- 각 직업-능력 조합이 **2개 행**으로 저장됨
  - 1행: Importance (IM) - 중요도 점수
  - 1행: Level (LV) - 수준 점수

**우리가 사용할 컬럼:**
- `O*NET-SOC Code`: 직업 코드
- `Title`: 직업명
- `Element Name`: 능력명
- `Scale ID`: IM/LV 구분
- `Data Value`: 점수

**총 레코드 수:** 92,976개 (894 직업 × 52 능력 × 2 척도)

In [ ]:
# 실습: Importance (IM)와 Level (LV) 데이터 분리

In [ ]:
# 실습: O*NET 능력명 통일

## 3. Importance와 Level 합치기

### 현재 문제

O*NET 데이터는 Importance와 Level이 각각 별도의 행으로 저장되어 있습니다.
아래 코드로 직접 확인해봅시다.

In [ ]:
# CEO의 'Written Expression' 능력 데이터 확인
# IM (Importance)와 LV (Level)이 별도의 행으로 저장됨
ceo_wrt = onet_abil[
    (onet_abil['O*NET-SOC Code'] == '11-1011.00') &
    (onet_abil['Element Name'] == 'Written Expression')
]
ceo_wrt[['O*NET-SOC Code', 'Element Name', 'Scale ID', 'Data Value']]

위 결과를 보면:
- **1번째 행**: Scale ID = 'IM', Data Value = 4.12 (중요도)
- **2번째 행**: Scale ID = 'LV', Data Value = 4.75(수준)

같은 직업-능력 정보가 2개 행으로 나뉘어 있습니다.

**목표:** 이 2개 행을 1개 행으로 합치기
→ 한 행에 Importance와 Level이 모두 있어야 계산하기 편함

In [ ]:
# 실습: Importance와 Level 병합

In [ ]:
# 실습: 컬럼명 정리 및 필요한 컬럼만 선택

## 4. Beta 값 추가하기

### .map() 함수란?

`.map()`은 딕셔너리를 사용해 Series의 값을 변환하는 메서드입니다.

**간단한 예제:**
```python
# 과일 가격 딕셔너리
prices = {'사과': 1000, '바나나': 500, '오렌지': 1500}

# 과일 리스트를 가격으로 변환
fruits = pd.Series(['사과', '바나나', '사과', '오렌지'])
fruit_prices = fruits.map(prices)
# 결과: [1000, 500, 1000, 1500]
```

**우리 경우:**
- `beta_values`: 능력명 → Beta 값 딕셔너리
- `onet_merged['Ability']`: 능력명 Series
- `.map(beta_values)`: 각 능력명을 Beta 값으로 변환

In [ ]:
# 실습: Beta 값을 onet_merged에 추가하세요

## 5. 최종 데이터 저장

In [ ]:
# 실습: ability_unified 컬럼 제거

In [ ]:
# 최종 데이터 저장
output_file = f'{PROCESSED_DIR}/preprocessed_data.csv'
onet_merged.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"✅ 전처리 완료! 저장 위치: {output_file}")
print(f"📊 최종 데이터 형태: {onet_merged.shape}")